Back to the main [Index](../index.ipynb)

# Creating an `AbinitInput` object

The creation of the Abinit input file is one of the most repetive and error-prone operations we have to do before running our calculations.
To facilitate the creation of the input file, one can use `AbinitInput`, a dict-like object 
that stores the Abinit variables and provides helper functions to automate the specification 
of multiple parameters

The code below shows how to create an `AbiniInput` and how to define the parameters of the calculation.

In [3]:
from __future__ import division, print_function, unicode_literals

import abipy.data as abidata
import abipy.abilab as abilab
from abipy.abilab import AbinitInput

# To create an ABINIT input we must specify the paths of the pseudopotential files.
# In this case, we have a pseudo named `14si.pspnc` located in the abidata.pseudo_dir directory.
inp = AbinitInput(structure=abidata.cif_file("si.cif"), 
                  pseudos="14si.pspnc", pseudo_dir=abidata.pseudo_dir)

In [4]:
# print will return a string with our input. In this case, the input is empty since 
# only the pseudos have been specified
print(inp)

############################################################################################
#                                         STRUCTURE                                         
############################################################################################
 acell    1.0    1.0    1.0
 xred
    0.0000000000    0.0000000000    0.0000000000
    0.2500000000    0.2500000000    0.2500000000
 rprim
    6.3285005272    0.0000000000    3.6537614829
    2.1095001757    5.9665675167    3.6537614829
    0.0000000000    0.0000000000    7.3075229659
 typat 1 1
 natom 2
 ntypat 1
 znucl 14


#<JSON>
#{
#    "pseudos": [
#        {
#            "l_max": 2, 
#            "Z_val": 4.0, 
#            "filepath": "/Users/gmatteo/git_repos/abipy/abipy/data/pseudos/14si.pspnc", 
#            "symbol": "Si", 
#            "@module": "pymatgen.io.abinit.pseudos", 
#            "Z": 14, 
#            "type": "NcAbinitPseudo", 
#            "basename": "14si.pspnc", 
#            "@class"

#### The function `set_vars` allows one to set the value of multiple variable with a single call

In [5]:
inp.set_vars(ecut=8, paral_kgb=0)

{'ecut': 8, 'paral_kgb': 0}

#### `AbinitInput` is a dict-like object, hence one can test for the presence of a variable in the input:

In [6]:
"ecut" in inp

True

#### List all the variables that have been defined:

In [7]:
list(inp.keys())

['ecut', 'paral_kgb']

Access the value of a particular variable with the syntax

In [8]:
inp["ecut"]

8

Iterate over keywords and values:

In [9]:
for varname, varvalue in inp.items(): 
    print(varname, "-->", varvalue)

ecut --> 8
paral_kgb --> 0


Use lists, tuples or numpy arrays when abinit expects arrays

In [10]:
inp.set_vars(kptopt=1, 
             ngkpt=[2, 2, 2], 
             nshiftk=2, 
             shiftk=[0.0, 0.0, 0.0, 0.5, 0.5, 0.5]  # 2 shifts in one list
            )

# It's possible to use strings but use them only for special cases such as:
inp["istwfk"] = "*1"
print(inp)

############################################################################################
#                                      SECTION: varbas                                      
############################################################################################
 ecut 8
 kptopt 1
 shiftk
    0.0    0.0    0.0
    0.5    0.5    0.5
 ngkpt 2 2 2
 nshiftk 2
############################################################################################
#                                      SECTION: vardev                                      
############################################################################################
 istwfk *1
############################################################################################
#                                      SECTION: varpar                                      
############################################################################################
 paral_kgb 0
#############################################

If you mistype the name of the variable, `AbinitInput` raises an error:

In [11]:
inp.set_vars(perl=0)

AbinitInputError: perl is not a valid ABINIT variable.
If the name is correct, try to remove ~/.abinit/abipy/abinit_vars.pickle
and rerun the code. If the problems persists, contact the abipy developers
or add the variable to ~abipy/data/variables/abinit_vars.json


# Defining the crystalline structure

The `set_structure` method sets the value of the ABINIT variables:
    
   * acell
   * rprim
   * ntypat
   * natom 
   * typat
   * znucl
   * xred
    
It is always a good idea to set the structure immediately after the creation of `AbinitInput`
because several methods use this information to faciliate the specification of other variables 
For example, the `set_kpath` method use the structure to generate the $k$-path for band structure calculations.

<div class="alert alert">
`typat` must be consistent with the list of pseudopotentials passed to `AbinitInput`
</div>

`set_structure` accepts either a dictionary with the ABINIT variables or an external file. 

### From dictionary:

In [12]:
structure = dict(
    ntypat=1,         
    natom=2,
    typat=[1, 1],
    znucl=14,
    acell=3*[10.217],
    rprim=[[0.0,  0.5,  0.5],   
           [0.5,  0.0,  0.5],
           [0.5,  0.5,  0.0]],
    xred=[[0.0 , 0.0 , 0.0],
          [0.25, 0.25, 0.25]]
)

inp = AbinitInput(structure, pseudos=abidata.pseudos("14si.pspnc"))

### From file

In [13]:
# from a CIF file
inp.set_structure(abidata.cif_file("si.cif"))

# or from a Netcdf file produced by ABINIT
inp.set_structure(abidata.ref_file("si_scf_GSR.nc"))

Structure Summary
Lattice
    abc : 3.8669746352064829 3.8669746352113457 3.8669746351893135
 angles : 60.000000000188479 60.000000000146869 60.000000000400874
 volume : 40.888292275933381
      A : 3.3488982698838332 0.0 1.9334873175946568
      B : 1.1162994232769716 3.157371568197199 1.9334873175946568
      C : 0.0 0.0 3.8669746351893135
PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Si (1.1163, 0.7893, 1.9335) [0.2500, 0.2500, 0.2500]

Supported formats include:

   * CIF
   * POSCAR/CONTCAR
   * CHGCAR 
   * LOCPOT,
   * vasprun.xml
   * CSSR 
   * ABINIT Netcdf 
   * pymatgen's JSON serialized structures

## From the Materials Project database:

In [14]:
# https://www.materialsproject.org/materials/mp-149/
inp.set_structure(abilab.Structure.from_material_id("mp-149"))

Structure Summary
Lattice
    abc : 3.8669746228495039 3.866974623775052 3.8669746200000001
 angles : 60.000000032293386 60.000000024375858 60.000000024168102
 volume : 40.888291888494884
      A : 3.3488982599999999 0.0 1.9334873100000001
      B : 1.11629942 3.1573715600000001 1.9334873100000001
      C : 0.0 0.0 3.8669746200000001
PeriodicSite: Si (3.9070, 2.7627, 6.7672) [0.8750, 0.8750, 0.8750]
PeriodicSite: Si (0.5581, 0.3947, 0.9667) [0.1250, 0.1250, 0.1250]

Note that you can avoid the call to `set_structure` if the `structure` argument is passed to 
`AbiInput`:

In [15]:
AbinitInput(structure=abidata.cif_file("si.cif"), pseudos=abidata.pseudos("14si.pspnc"))

<AbinitInput at 4578295120>

## Brillouin zone sampling

There are two different types of sampling of the BZ: homogeneous and high-symmetry k-path.
The later is mainly used for band structure calculations and requires the specification of: 

   * kptopt
   * kptbounds
   * ndivsm
    
whereas the homogeneous sampling is needed for all the calculations in which 
we have to compute integrals in the Brillouin zone e.g. total energy calculations, DOS, etc.
The $k$-mesh is usually specified via:

   * ngkpt
   * nshiftk
   * shiftk

### Explicit $k$-mesh

In [16]:
inp = AbinitInput(structure=abidata.cif_file("si.cif"), pseudos=abidata.pseudos("14si.pspnc"))

# Set ngkpt, shiftk explictly  
inp.set_kmesh(ngkpt=(1, 2, 3), shiftk=[0.0, 0.0, 0.0, 0.5, 0.5, 0.5])

{'kptopt': 1,
 'ngkpt': (1, 2, 3),
 'nshiftk': 2,
 'shiftk': array([[ 0. ,  0. ,  0. ],
        [ 0.5,  0.5,  0.5]])}

### Automatic $k$-mesh 

In [17]:
# Define a homogeneous k-mesh. 
# nksmall is the number of divisions to be used to sample the smallest lattice vector,
# shiftk is automatically selected from an internal database.
inp.set_autokmesh(nksmall=4)

{'kptopt': 1,
 'ngkpt': array([4, 4, 4]),
 'nshiftk': 4,
 'shiftk': array([[ 0.5,  0.5,  0.5],
        [ 0.5,  0. ,  0. ],
        [ 0. ,  0.5,  0. ],
        [ 0. ,  0. ,  0.5]])}

### High-symmetry $k$-path

In [18]:
# Generate a high-symmetry k-path (taken from an internal database)
# 2 points are used to sample the smallest segment, 
# the other segments are sampled so that proportions are preserved.
inp.set_kpath(ndivsm=10)

{'iscf': -2, 'kptbounds': array([[ 0.   ,  0.   ,  0.   ],
        [ 0.5  ,  0.   ,  0.5  ],
        [ 0.5  ,  0.25 ,  0.75 ],
        [ 0.375,  0.375,  0.75 ],
        [ 0.   ,  0.   ,  0.   ],
        [ 0.5  ,  0.5  ,  0.5  ],
        [ 0.625,  0.25 ,  0.625],
        [ 0.5  ,  0.25 ,  0.75 ],
        [ 0.5  ,  0.5  ,  0.5  ],
        [ 0.375,  0.375,  0.75 ],
        [ 0.625,  0.25 ,  0.625],
        [ 0.5  ,  0.   ,  0.5  ]]), 'kptopt': -11, 'ndivsm': 10}

# Utilities

### Once the structure has been defined, one can compute the number of valence electrons with:

In [19]:
print("The number of valence electrons is: ", inp.num_valence_electrons)

The number of valence electrons is:  8.0


### Generating inputs for convergence studies

In [20]:
# When using a non-integer step, such as 0.1, the results will often not
# be consistent.  It is better to use ``linspace`` for these cases.
# See also numpy.arange and numpy.linspace
ecut_inps = inp.arange("ecut", start=2, stop=5, step=2)

print([i["ecut"] for i in ecut_inps])

[2, 4]


In [21]:
tsmear_inps = inp.linspace("tsmear", start=0.001, stop=0.003, num=3)
print([i["tsmear"] for i in tsmear_inps])

[0.001, 0.002, 0.0030000000000000001]


# Invoking Abinit with AbinitInput

Once you have an `AbinitInput` you can call Abinit to get useful information 
or simply to validate the input file before running the real calculation.
All the method that invoke Abinit starts with `abi` e.g. `abiget` or `abivalidate`.

In [22]:
inp = AbinitInput(structure=abidata.cif_file("si.cif"), pseudos=abidata.pseudos("14si.pspnc"))

inp.set_vars(ecut=-2)
inp.set_autokmesh(nksmall=4)

v = inp.abivalidate() 
if v.retcode != 0: 
    # If there's a mistake in the input, one can acces the log file of the run with:
    print(v.log_file.read())

  ABINIT 8.3.1
  
  Give name for formatted input file: 
/var/folders/89/47k8wfdj11x035svqf8qnl4m0000gn/T/tmpPVh_EB/run.abi
  Give name for formatted output file:
/var/folders/89/47k8wfdj11x035svqf8qnl4m0000gn/T/tmpPVh_EB/run.abo
  Give root name for generic input files:
/var/folders/89/47k8wfdj11x035svqf8qnl4m0000gn/T/tmpPVh_EB/indata/in
  Give root name for generic output files:
/var/folders/89/47k8wfdj11x035svqf8qnl4m0000gn/T/tmpPVh_EB/outdata/out
  Give root name for generic temporary files:
/var/folders/89/47k8wfdj11x035svqf8qnl4m0000gn/T/tmpPVh_EB/tmpdata/tmp

.Version 8.3.1 of ABINIT 
.(MPI version, prepared for a x86_64_darwin16.3.0_gnu6.0 computer) 

.Copyright (C) 1998-2016 ABINIT group . 
 ABINIT comes with ABSOLUTELY NO WARRANTY.
 It is free software, and you are welcome to redistribute it
 under certain conditions (GNU General Public License,
 see ~abinit/COPYING or http://www.gnu.org/copyleft/gpl.txt).

 ABINIT is a project of the Universite Catholique de Louvain,
 Cornin

In [30]:
# Fix the problem with the negative ecut and rerun validate!
inp["ecut"] = 2
inp["toldfe"] = 1e-10
v = inp.abivalidate()
if v.retcode == 0: 
    print("All ok")
else:
    print(v)

All ok


At this point, we have a valid input file and we can get the k-points in the irreducible zone with:

In [31]:
ibz = inp.abiget_ibz()
print("number of k-points:", len(ibz.points))
print("k-points:", ibz.points)
print("weights:", ibz.weights)
print("weights are normalized to:", ibz.weights.sum())

number of k-points: 10
k-points: [[-0.125 -0.25   0.   ]
 [-0.125  0.5    0.   ]
 [-0.25  -0.375  0.   ]
 [-0.125 -0.375  0.125]
 [-0.125  0.25   0.   ]
 [-0.25   0.375  0.   ]
 [-0.375  0.5    0.   ]
 [-0.25   0.5    0.125]
 [-0.125  0.     0.   ]
 [-0.375  0.     0.   ]]
weights: [ 0.09375  0.09375  0.09375  0.1875   0.09375  0.09375  0.09375  0.1875
  0.03125  0.03125]
weights are normalized to: 1.0


In [32]:
# To get the list of possible parallel configurations for this input up to 5 max_ncpus
inp["paral_kgb"] = 1
pconfs = inp.abiget_autoparal_pconfs(max_ncpus=5)

In [33]:
print("best efficiency:\n", pconfs.sort_by_efficiency()[0])
print("best speedup:\n", pconfs.sort_by_speedup()[0])

best efficiency:
 {'efficiency': 0.975,
 u'mem_per_cpu': 0.0,
 'mpi_ncpus': 5,
 u'omp_ncpus': 1,
 'tot_ncpus': 5,
 'vars': {'bandpp': 1,
          'npband': 1,
          'npfft': 1,
          'npimage': 1,
          'npkpt': 5,
          'npspinor': 1}}

best speedup:
 {'efficiency': 0.975,
 u'mem_per_cpu': 0.0,
 'mpi_ncpus': 5,
 u'omp_ncpus': 1,
 'tot_ncpus': 5,
 'vars': {'bandpp': 1,
          'npband': 1,
          'npfft': 1,
          'npimage': 1,
          'npkpt': 5,
          'npspinor': 1}}



In [34]:
# To get the list of irreducible phonon perturbations at Gamma (Abinit notation)
inp.abiget_irred_phperts(qpt=(0, 0, 0))

[{'idir': 1, 'ipert': 1, 'qpt': [0.0, 0.0, 0.0]}]

# Multiple datasets

Multiple datasets are handy when you have to generate several input files sharing several common
variables e.g. the crystalline structure, the smearing value etc...
In this case, one can use the `MultiDataset` object that is essentially 
a list of `AbinitInput` objects.

In [35]:
# A MultiDataset object with two datasets (a.k.a. AbinitInput) 
multi = abilab.MultiDataset(structure=abidata.cif_file("si.cif"),
                            pseudos="14si.pspnc", pseudo_dir=abidata.pseudo_dir, ndtset=2)

# A MultiDataset is essentially a list if AbinitInput objects 
# with handy methods to perform global modifications.
# i.e. changes that will affect all the inputs in the MultiDataset
# For example:
multi.set_vars(ecut=4)

# is equivalent to
#
#   for inp in multi: inp.set_vars(ecut=4)
#
# and indeed:

for inp in multi: 
    print(inp["ecut"])

4
4


In [36]:
# To change the values in a particular dataset use:
multi[0].set_vars(ngkpt=[2,2,2], tsmear=0.004)
multi[1].set_vars(ngkpt=[4,4,4], tsmear=0.008)

print(multi)

ndtset 2

#################
### DATASET 1 ###
#################
############################################################################################
#                                      SECTION: varbas                                      
############################################################################################
 ecut1 4
 ngkpt1 2 2 2
############################################################################################
#                                       SECTION: vargs                                      
############################################################################################
 tsmear1 0.004
############################################################################################
#                                         STRUCTURE                                         
############################################################################################
 acell1    1.0    1.0    1.0
 xred1
    0.0000000000    0.000

<div class="alert alert">
In python we start to count from zero hence the first dataset has index 0.
</div>

In [37]:
# Calling set_structure on MultiDataset will set the structure of the inputs
multi.set_structure(abidata.cif_file("si.cif"))

# The structure attribute of a MultiDataset returns a list of structures 
# equivalent to [inp.structure for inp in multi]
multi.structure

[Structure Summary
 Lattice
     abc : 3.8669746200000001 3.8669746200000001 3.8669746200000001
  angles : 59.999999999999993 59.999999999999993 59.999999999999993
  volume : 40.888291793468909
       A : 3.3488982567096763 0.0 1.9334873100000005
       B : 1.1162994189032256 3.1573715557642927 1.9334873100000005
       C : 0.0 0.0 3.8669746200000001
 PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
 PeriodicSite: Si (1.1163, 0.7893, 1.9335) [0.2500, 0.2500, 0.2500],
 Structure Summary
 Lattice
     abc : 3.8669746200000001 3.8669746200000001 3.8669746200000001
  angles : 59.999999999999993 59.999999999999993 59.999999999999993
  volume : 40.888291793468909
       A : 3.3488982567096763 0.0 1.9334873100000005
       B : 1.1162994189032256 3.1573715557642927 1.9334873100000005
       C : 0.0 0.0 3.8669746200000001
 PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
 PeriodicSite: Si (1.1163, 0.7893, 1.9335) [0.2500, 0.2500, 0.2500]]

The function `split_datasets` return the list of `AbinitInput` stored in MultiDataset

In [38]:
inp0, inp1 = multi.split_datasets()
print(inp0)

############################################################################################
#                                      SECTION: varbas                                      
############################################################################################
 ecut 4
 ngkpt 2 2 2
############################################################################################
#                                       SECTION: vargs                                      
############################################################################################
 tsmear 0.004
############################################################################################
#                                         STRUCTURE                                         
############################################################################################
 acell    1.0    1.0    1.0
 xred
    0.0000000000    0.0000000000    0.0000000000
    0.2500000000    0.2500000000    0.2500000

<div class="alert alert">
You can use `MultiDataset` to build your input files but remember that 
`Abipy` will never support input files with more than one dataset.
As a consequence, you should always pass an `AbinitInput` to the 
AbiPy functions that are building `Tasks`, `Works` or `Flows`.
</div>

In [39]:
print("Number of datasets:", multi.ndtset)

Number of datasets: 2


In [40]:
# To create and append a new dataset (initialized from dataset number 1)
multi.addnew_from(1)
multi[-1].set_vars(ecut=42)
print("Now multi has", multi.ndtset, "datasets and the ecut in the last dataset is:", 
      multi[-1]["ecut"])

Now multi has 3 datasets and the ecut in the last dataset is: 42


# Factory functions

TODO

# Decorators

TODO